<a href="https://colab.research.google.com/github/Pathway2008/Weblog/blob/main/comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/Weblog/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Weblog/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/Weblog/sample_submission.csv')

In [2]:
train.drop(['sessionID','userID'],axis=1,inplace=True)
test.drop(['sessionID','userID'],axis=1,inplace=True)

In [3]:
import warnings
import matplotlib.font_manager as fm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
warnings.filterwarnings('ignore')
!apt-get -qq install fonts-nanum


fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
    name='NanumGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})

Selecting previously unselected package fonts-nanum.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [4]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
object_columns = train.drop(columns=['TARGET']).select_dtypes(include='object')
for feature in object_columns.columns:
    le = LabelEncoder()

    # Convert the column to string type
    train[feature] = train[feature].astype(str)
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    # Make sure the test set has the same data type
    test[feature] = test[feature].astype(str)

    # Handle unseen labels in the test set
    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)

    test[feature] = le.transform(test[feature])

In [5]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 16.2 MB/s eta 0:00:00


In [6]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from math import sqrt
from sklearn.metrics import make_scorer, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [7]:
from sklearn.model_selection import KFold
kf = StratifiedKFold(n_splits=5, shuffle=False)

In [8]:
def rmse_scorer(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [29]:
from sklearn.model_selection import train_test_split
X = train.drop(['TARGET','keyword' ,'traffic_medium','device'], axis=1)
y = train['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 43)

In [30]:
# Random Forest
rf_model = RandomForestRegressor(random_state=43)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
rf_rmse = rmse_scorer(y_test, rf_predictions)


# Extra Trees
et_model = ExtraTreesRegressor(random_state=43)
et_model.fit(X_train, y_train)
et_predictions = et_model.predict(X_test)
et_scores = cross_val_score(et_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
et_rmse = rmse_scorer(y_test, et_predictions)

#xgb
xgb_model = xgb.XGBRegressor(random_state=43, reg_lambda=4)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
xgb_rmse = rmse_scorer(y_test, xgb_predictions)

#cat
cat_model = CatBoostRegressor(random_seed=43, l2_leaf_reg=4, verbose=0)
cat_model.fit(X_train, y_train)
cat_predictions = cat_model.predict(X_test)
cat_scores = cross_val_score(cat_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
cat_rmse = rmse_scorer(y_test, cat_predictions)

#lgb
lgb_model = LGBMRegressor(random_state=43, reg_lambda=4)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)
lgb_scores = cross_val_score(lgb_model, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))
lgb_rmse = rmse_scorer(y_test, lgb_predictions)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007236 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1236
[LightGBM] [Info] Number of data points in the train set: 176602, number of used features: 13
[LightGBM] [Info] Start training from score 3.593011
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004722 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1218
[LightGBM] [Info] Number of data points in the train set: 141281, number of used features: 13
[LightGBM] [Info] Start training from score 3.593406
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005573 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not eno

In [14]:
# basic
print(f'Random Forest Cross-Validation RMSE: {rf_scores.mean()}')
print(f'RF test RMSE: {rf_rmse}\n')
print(f'Extra Trees Cross-Validation RMSE: {et_scores.mean()}')
print(f'ET test RMSE: {et_rmse}\n')
print(f'XGBoost Cross-Validation RMSE: {xgb_scores.mean()}')
print(f'XGBoost test RMSE: {xgb_rmse}\n')
print(f'CatBoost Cross-Validation RMSE: {cat_scores.mean()}')
print(f'CatBoost test RMSE: {cat_rmse}\n')
print(f"Cross-validated LightGBM scores: {lgb_scores.mean()}")
print(f"LightGBM RMSE on test set: {lgb_rmse}")

Random Forest Cross-Validation RMSE: 2.8772603501599288
RF test RMSE: 2.787345470753536

Extra Trees Cross-Validation RMSE: 2.9151268400916703
ET test RMSE: 2.9101033585414218

XGBoost Cross-Validation RMSE: 2.8491038194831644
XGBoost test RMSE: 2.745348382572756

CatBoost Cross-Validation RMSE: 2.7631369388930205
CatBoost test RMSE: 2.7357731787807276

Cross-validated LightGBM scores: 2.750722150406905
LightGBM RMSE on test set: 2.7743799222694636


In [22]:
# drop 'keyword' ,'traffic_medium','device'
print(f'Random Forest Cross-Validation RMSE: {rf_scores.mean()}')
print(f'RF test RMSE: {rf_rmse}\n')
print(f'Extra Trees Cross-Validation RMSE: {et_scores.mean()}')
print(f'ET test RMSE: {et_rmse}\n')
print(f'XGBoost Cross-Validation RMSE: {xgb_scores.mean()}')
print(f'XGBoost test RMSE: {xgb_rmse}\n')
print(f'CatBoost Cross-Validation RMSE: {cat_scores.mean()}')
print(f'CatBoost test RMSE: {cat_rmse}\n')
print(f"Cross-validated LightGBM scores: {lgb_scores.mean()}")
print(f"LightGBM RMSE on test set: {lgb_rmse}")

Random Forest Cross-Validation RMSE: 2.8670504774502477
RF test RMSE: 2.812503055608758

Extra Trees Cross-Validation RMSE: 2.9281515668953237
ET test RMSE: 2.921826705209167

XGBoost Cross-Validation RMSE: 2.8687832089766863
XGBoost test RMSE: 2.773079692905336

CatBoost Cross-Validation RMSE: 2.7775146436911076
CatBoost test RMSE: 2.7646066612403573

Cross-validated LightGBM scores: 2.751366832121126
LightGBM RMSE on test set: 2.7803121117453795


In [31]:
from sklearn.ensemble import VotingRegressor
voting_regressor = VotingRegressor([('xgb', xgb_model), ('catboost', cat_model),('lgb',lgb_model),('et',et_model),('rf',rf_model)])
#   #cat_model로 수정필요,  weights=[1.5, 1.5, 1, 1, 1.5]
voting_regressor.fit(X_train, y_train)

voting_predictions = voting_regressor.predict(X_test)

voting_rmse = rmse_scorer(y_test, voting_predictions)
voting_scores = cross_val_score(voting_regressor, X_train, y_train, cv=kf, scoring=make_scorer(rmse_scorer))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006059 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1236
[LightGBM] [Info] Number of data points in the train set: 176602, number of used features: 13
[LightGBM] [Info] Start training from score 3.593011
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006013 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1218
[LightGBM] [Info] Number of data points in the train set: 141281, number of used features: 13
[LightGBM] [Info] Start training from score 3.593406
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004749 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not eno

In [16]:
print(f'VotingRegressor Cross-Validation RMSE: {voting_scores.mean()}')
print(f'VotingRegressor test RMSE: {voting_rmse}')

VotingRegressor Cross-Validation RMSE: 2.721710996963167
VotingRegressor test RMSE: 2.6907786729299086


In [28]:
# drop 'keyword' ,'traffic_medium','device'
print(f'VotingRegressor Cross-Validation RMSE: {voting_scores.mean()}')
print(f'VotingRegressor test RMSE: {voting_rmse}')

VotingRegressor Cross-Validation RMSE: 2.725913937034029
VotingRegressor test RMSE: 2.707253121736505


In [32]:
# drop 'keyword' ,'traffic_medium','device' test
print(f'VotingRegressor Cross-Validation RMSE: {voting_scores.mean()}')
print(f'VotingRegressor test RMSE: {voting_rmse}')

VotingRegressor Cross-Validation RMSE: 2.725913937034029
VotingRegressor test RMSE: 2.707253121736505


In [17]:
# Random Forest
rf_model = RandomForestRegressor(random_state=43)
rf_model.fit(X_train.drop(['browser','bounced','device'],axis=1), y_train)
rf_predictions = rf_model.predict(X_test.drop(['browser','bounced','device'],axis=1))
rf_scores = cross_val_score(rf_model, X_train.drop(['browser','bounced','device'],axis=1), y_train, cv=kf, scoring=make_scorer(rmse_scorer))
rf_rmse = rmse_scorer(y_test, rf_predictions)


# Extra Trees
et_model = ExtraTreesRegressor(random_state=43)
et_model.fit(X_train.drop(['device', 'keyword', 'transaction_revenue'],axis=1), y_train)
et_predictions = et_model.predict(X_test.drop(['device', 'keyword', 'transaction_revenue'],axis=1))
et_scores = cross_val_score(et_model, X_train.drop(['device', 'keyword', 'transaction_revenue'],axis=1), y_train, cv=kf, scoring=make_scorer(rmse_scorer))
et_rmse = rmse_scorer(y_test, et_predictions)

#xgb
xgb_model = xgb.XGBRegressor(random_state=43, reg_lambda=4)
xgb_model.fit(X_train.drop(['device', 'subcontinent', 'keyword'],axis=1), y_train)
xgb_predictions = xgb_model.predict(X_test.drop(['device', 'subcontinent', 'keyword'],axis=1))
xgb_scores = cross_val_score(xgb_model, X_train.drop(['device', 'subcontinent', 'keyword'],axis=1), y_train, cv=kf, scoring=make_scorer(rmse_scorer))
xgb_rmse = rmse_scorer(y_test, xgb_predictions)

xgb_model2 = xgb.XGBRegressor(random_state=43, reg_lambda=4)
xgb_model2.fit(X_train.drop(['device', 'transaction', 'traffic_medium'],axis=1), y_train)
xgb_predictions2 = xgb_model2.predict(X_test.drop(['device', 'transaction', 'traffic_medium'],axis=1))
xgb_scores2 = cross_val_score(xgb_model2, X_train.drop(['device', 'transaction', 'traffic_medium'],axis=1), y_train, cv=kf, scoring=make_scorer(rmse_scorer))
xgb_rmse2 = rmse_scorer(y_test, xgb_predictions2)

#cat
cat_model = CatBoostRegressor(random_seed=43, l2_leaf_reg=4, verbose=0)
cat_model.fit(X_train.drop(['browser', 'OS', 'bounced'],axis=1), y_train)
cat_predictions = cat_model.predict(X_test.drop(['browser', 'OS', 'bounced'],axis=1))
cat_scores = cross_val_score(cat_model, X_train.drop(['browser', 'OS', 'bounced'],axis=1), y_train, cv=kf, scoring=make_scorer(rmse_scorer))
cat_rmse = rmse_scorer(y_test, cat_predictions)

#lgb
lgb_model = LGBMRegressor(random_state=43, reg_lambda=4)
lgb_model.fit(X_train.drop(['device', 'keyword', 'transaction_revenue', 'subcontinent'],axis=1), y_train)
lgb_predictions = lgb_model.predict(X_test.drop(['device', 'keyword', 'transaction_revenue', 'subcontinent'],axis=1))
lgb_scores = cross_val_score(lgb_model, X_train.drop(['device', 'keyword', 'transaction_revenue', 'subcontinent'],axis=1), y_train, cv=kf, scoring=make_scorer(rmse_scorer))
lgb_rmse = rmse_scorer(y_test, lgb_predictions)

#lgb
lgb_model2 = LGBMRegressor(random_state=43, reg_lambda=4)
lgb_model2.fit(X_train.drop(['device', 'browser', 'traffic_medium', 'referral_path'],axis=1), y_train)
lgb_predictions2 = lgb_model2.predict(X_test.drop(['device', 'browser', 'traffic_medium', 'referral_path'],axis=1))
lgb_scores2 = cross_val_score(lgb_model2, X_train.drop(['device', 'browser', 'traffic_medium', 'referral_path'],axis=1), y_train, cv=kf, scoring=make_scorer(rmse_scorer))
lgb_rmse2 = rmse_scorer(y_test, lgb_predictions2)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005862 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 965
[LightGBM] [Info] Number of data points in the train set: 176602, number of used features: 12
[LightGBM] [Info] Start training from score 3.593011
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004517 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 946
[LightGBM] [Info] Number of data points in the train set: 141281, number of used features: 12
[LightGBM] [Info] Start training from score 3.593406
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005262 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

In [18]:
# Optimization
print(f'Random Forest Cross-Validation RMSE: {rf_scores.mean()}')
print(f'RF test RMSE: {rf_rmse}\n')
print(f'Extra Trees Cross-Validation RMSE: {et_scores.mean()}')
print(f'ET test RMSE: {et_rmse}\n')
print(f'XGBoost Cross-Validation RMSE: {xgb_scores.mean()}')
print(f'XGBoost test RMSE: {xgb_rmse}\n')
print(f'XGBoost Cross-Validation RMSE: {xgb_scores2.mean()}')
print(f'XGBoost test RMSE: {xgb_rmse2}\n')
print(f'CatBoost Cross-Validation RMSE: {cat_scores.mean()}')
print(f'CatBoost test RMSE: {cat_rmse}\n')
print(f"Cross-validated LightGBM scores: {lgb_scores.mean()}")
print(f"LightGBM RMSE on test set: {lgb_rmse}\n")
print(f"Cross-validated LightGBM scores: {lgb_scores2.mean()}")
print(f"LightGBM RMSE on test set: {lgb_rmse2}")

Random Forest Cross-Validation RMSE: 2.898682113834506
RF test RMSE: 2.840535879997574

Extra Trees Cross-Validation RMSE: 2.932161278546251
ET test RMSE: 2.9187279938954465

XGBoost Cross-Validation RMSE: 2.8544205391339
XGBoost test RMSE: 2.7550007033393524

XGBoost Cross-Validation RMSE: 2.8842926588142146
XGBoost test RMSE: 2.7609070243830485

CatBoost Cross-Validation RMSE: 2.8127841597219634
CatBoost test RMSE: 2.8296627668725565

Cross-validated LightGBM scores: 2.761286572480066
LightGBM RMSE on test set: 2.7674292612011238

Cross-validated LightGBM scores: 2.759969462055403
LightGBM RMSE on test set: 2.7899963066913798


In [ ]:
# Random Forest Cross-Validation RMSE: 2.8772603501599288
# RF test RMSE: 2.787345470753536

# Extra Trees Cross-Validation RMSE: 2.9151268400916703
# ET test RMSE: 2.9101033585414218

# XGBoost Cross-Validation RMSE: 2.8491038194831644
# XGBoost test RMSE: 2.745348382572756

# CatBoost Cross-Validation RMSE: 2.7631369388930205
# CatBoost test RMSE: 2.7357731787807276

# Cross-validated LightGBM scores: 2.750722150406905
# LightGBM RMSE on test set: 2.7743799222694636

In [19]:
rmse_mean = rmse_scorer(y_test, (lgb_predictions + rf_predictions + et_predictions + xgb_predictions + cat_predictions+
                                 xgb_predictions2 + lgb_predictions2)/6)
rmse_mean

2.8611968648301067